# Cube and Data Warehouse

In [23]:
import atoti as tt

In [24]:
session = tt.Session(
    user_content_storage=".content",
    port=9092,
    java_options=["-Xms1G", "-Xmx10G"]
)

Closing existing "Unnamed" session to create the new one.


In [25]:
db_name = "olympic_olap"
db_user = "postgres"
db_password = "postgres"  # Update with your password
db_host = "pgdb"  # Update if your DB is hosted elsewhere
db_port = "5432"

jdbc_url = f"jdbc:postgresql://{db_host}:{db_port}/{db_name}?user={db_user}&password={db_password}"

In [26]:
olympic_medals_measures = session.read_sql(
    "SELECT * FROM factolympicmedalsmeasures",
    table_name="FactOlympicMedalsMeasures",
    url=jdbc_url,
)

In [27]:
dimlocation_table = session.read_sql(
    "SELECT * FROM DimLocation",
    keys=["country_code"],
    table_name="DimLocation",
    url=jdbc_url)

In [28]:
dimevent_table = session.read_sql(
    "SELECT * FROM DimEvent",
    keys=["event_id"],
    table_name="DimEvent",
    url=jdbc_url,
)

In [29]:
dimparticipant_table = session.read_sql(
    "SELECT * FROM DimParticipant",
    keys=["participant_id"],
    table_name="DimParticipant",
    url=jdbc_url,
)

In [30]:
dimathlete_table = session.read_sql(
    "SELECT * FROM DimAthlete",
    keys=["athlete_id"],
    table_name="DimAthelete",
    url=jdbc_url,
)

In [31]:
dimyear_table = session.read_sql(
    "SELECT * FROM DimYear",
    keys=["year"],
    table_name="DimYear",
    url=jdbc_url,
)

In [32]:
dimgame_table = session.read_sql(
    "SELECT * FROM DimGame",
    keys=["game_slug"],
    table_name="DimGame",
    url=jdbc_url,
)

In [33]:
olympic_medals_measures.join(dimlocation_table, olympic_medals_measures["country_code"] == dimlocation_table["country_code"])

olympic_medals_measures.join(dimevent_table, olympic_medals_measures["event_id"] == dimevent_table["event_id"])

olympic_medals_measures.join(dimparticipant_table, olympic_medals_measures["participant_id"] == dimparticipant_table["participant_id"])

olympic_medals_measures.join(dimathlete_table, olympic_medals_measures["athlete_id"] == dimathlete_table["athlete_id"])

olympic_medals_measures.join(dimyear_table, olympic_medals_measures["year"] == dimyear_table["year"])

olympic_medals_measures.join(dimgame_table, olympic_medals_measures["game_slug"] == dimgame_table["game_slug"])

In [34]:
health_measures  = session.read_sql(
    "SELECT * FROM facthealthmeasure",
    table_name="FactHealthMeasure",
    url=jdbc_url,
)



In [35]:
health_measures.join(dimlocation_table, health_measures["country_code"] == dimlocation_table["country_code"])

In [36]:
health_measures.join(dimyear_table, health_measures["year"] == dimyear_table["year"])

In [37]:
economic_measures = session.read_sql(
    "SELECT * FROM facteconomicmeasure",
    table_name="FactEconomicMeasure",
    url=jdbc_url,
)

In [38]:
economic_measures.join(dimyear_table, economic_measures["year"] == dimyear_table["year"])

In [39]:
economic_measures.join(dimlocation_table, economic_measures["country_code"] == dimlocation_table["country_code"])

In [40]:
session.tables.schema

```mermaid
erDiagram
  "DimLocation" {
    _ String PK "country_code"
    _ String "country_name"
    _ String "continent"
  }
  "FactHealthMeasure" {
    nullable int "year"
    _ String "country_code"
    nullable double "daly_depression"
    nullable double "daly_schizophrenia"
    nullable double "daly_bipolar_disorder"
    nullable double "daly_eating_disorder"
    nullable double "daly_anxiety"
    nullable double "life_expectancy"
    nullable double "infant_mortality_rate"
    nullable double "current_health_expenditure"
    nullable double "government_health_expenditure"
    nullable double "private_health_expenditure"
    nullable double "external_health_expenditure"
  }
  "FactEconomicMeasure" {
    nullable int "year"
    _ String "country_code"
    nullable double "poverty_count"
    nullable double "gdp_per_capita"
    nullable double "annual_gdp_growth"
    nullable int "servers_count"
  }
  "DimEvent" {
    _ int PK "event_id"
    _ String "event_title"
    _ String "event_discipline"
    _ String "event_gender"
  }
  "DimParticipant" {
    _ int PK "participant_id"
    _ String "participant_title"
    _ String "participant_type"
  }
  "DimAthelete" {
    _ int PK "athlete_id"
    _ String "athlete_name"
    _ String "athlete_url"
  }
  "DimYear" {
    _ int PK "year"
  }
  "DimGame" {
    _ String PK "game_slug"
    _ String "game_name"
    _ String "game_season"
    nullable int "game_year"
    _ String "country_code"
  }
  "FactOlympicMedalsMeasures" {
    _ String "game_slug"
    nullable int "participant_id"
    nullable int "athlete_id"
    nullable int "event_id"
    _ String "country_code"
    nullable int "year"
    nullable int "total_bronze_medals"
    nullable int "total_silver_medals"
    nullable int "total_gold_medals"
    nullable int "total_medals"
  }
  "FactHealthMeasure" }o--o| "DimLocation" : "`country_code` == `country_code`"
  "FactHealthMeasure" }o--o| "DimYear" : "`year` == `year`"
  "FactEconomicMeasure" }o--o| "DimLocation" : "`country_code` == `country_code`"
  "FactEconomicMeasure" }o--o| "DimYear" : "`year` == `year`"
  "FactOlympicMedalsMeasures" }o--o| "DimGame" : "`game_slug` == `game_slug`"
  "FactOlympicMedalsMeasures" }o--o| "DimLocation" : "`country_code` == `country_code`"
  "FactOlympicMedalsMeasures" }o--o| "DimAthelete" : "`athlete_id` == `athlete_id`"
  "FactOlympicMedalsMeasures" }o--o| "DimEvent" : "`event_id` == `event_id`"
  "FactOlympicMedalsMeasures" }o--o| "DimYear" : "`year` == `year`"
  "FactOlympicMedalsMeasures" }o--o| "DimParticipant" : "`participant_id` == `participant_id`"
```


In [ ]:
# # Define outer joins
# # Test with a single join condition
# olympic_medals_measures.join(health_measures, olympic_medals_measures["year"] == health_measures["year"])


In [18]:
cube = session.create_cube(olympic_medals_measures)

In [19]:
cube

In [20]:
hierarchies, levels, measures = cube.hierarchies, cube.levels, cube.measures

In [21]:
levels

In [22]:
hierarchies["DimAthlete"] = [levels["athlete_name"]]

In [23]:
hierarchies["DimEvent"] = [levels["event_title"], levels[("DimEvent", "event_discipline", "event_discipline")],
                           levels[("DimEvent", "event_gender", "event_gender")]]